# Take Control of GPT : Use the API. Don't let the API use you

### My first openai API Project!!!

Here, I use the openai Python lib and some custom classes to handle processing prompts and responses. I store the results in a pandas dataframe, then format the code in a pandas 'style' object. I can use the dataframe and other objects (like the multiline string of prompts) to manage context for later prompts. Yes, context from prompts and answers. So much more to do with this. But this is a start.

## Setup

In [14]:
%%html
<style>
    /* Enable word wrap */
    .rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
      white-space: nowrap;
      overflow: hidden;
      text-overflow: ellipsis;
    }
</style>

### Imports

In [15]:
# ...
import openai
import os
import pprint
import sys
from pprint import pprint
import json
import pyarrow as pa
import pandas as pd

lib = os.path.abspath(os.path.join('..', '..', 'libs'))
sys.path.append(lib)

from MessageParser import MessageParser
from ResponseParser import ResponseParser
from PromptLogs import PromptLogs


### Config

In [16]:
# ...
# *******************************************************************************************************
# SET THE API KEY
# ================
# -- set the env variable OPENAI_API_KEY to your OpenAI API key       <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
openai.api_key = os.getenv("OPENAI_API_KEY")
# *******************************************************************************************************

# Setup up pandas properties
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line break in DataFrame
pd.set_option('display.max_colwidth', None)  # Display the full content of cells



## Main code

Prep the system prompts

In [17]:
sys_prompt_list = []
sys_prompt_1 = """You are a software expert."""
sys_prompt_2 = """Use. "```python" style for code blocks."""
sys_prompt_3 = """Include the name for a file where I would store individual code for functions or classes."""

sys_prompt_list.append(sys_prompt_1)
sys_prompt_list.append(sys_prompt_2)
sys_prompt_list.append(sys_prompt_3)

sys_prompt_string = "".join(sys_prompt_list)

### a PromptLogs object will hold all the individual individual parser objects, which hold the response from a single exchange

In [18]:
logs = PromptLogs()

Generate the message and make the call

In [19]:
message_list = f"""
system: {sys_prompt_string}
user: Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.
"""

# a MessageParser object will prcoess the message_list and make an acceptable messages object
response_parser = MessageParser(message_list)
messages = response_parser.parse_messages()

#  make the openai call and get the response
# I'll abstract this away in the future
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)


A response parser will process the response

In [20]:
response_parser = ResponseParser(response=response)

See a pandas dataframe of the logged response

In [28]:
response_parser.df

,id,created,model,usage,all_code,code_blocks,content
0,chatcmpl-73ywxpWRLcU2BzzKGkaMBluPZBmqn,1.681184e+09,gpt-3.5-turbo-0301,60.0,"def&nbsp;get_numbers():<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;[num&nbsp;for&nbsp;num&nbsp;in&nbsp;range(1,11)]<br>from&nbsp;utilities&nbsp;import&nbsp;get_numbers<br>","[\ndef get_numbers():\n return [num for num in range(1,11)]\n, \nfrom utilities import get_numbers\n]","The function to return numbers 1-10 in a list would look like this:\n\n```python\ndef get_numbers():\n return [num for num in range(1,11)]\n```\n\nHere, we have defined a function called `get_numbers` which simply returns a list containing numbers from 1 to 10. We're using a list comprehension to create and return the list. Note that we specify the range(1, 11), as it includes all values from 1 to 10. \n\nYou can store this function in a file called `utilities.py`. Of course, you could also use another name for the file as well. You'd just need to use the appropriate file name when you import the function elsewhere in your project. The import statement would look something like this:\n\n```python\nfrom utilities import get_numbers\n```"


In [22]:
# A pandas style applies custom formatting to a DataFrame to make it easier to read and
# understand. It does not offer the full range of functionality that a DataFrame provides.
response_parser.styled_log

,id,created,model,usage,all_code
0,chatcmpl-73ywrIJ5cfirs5nwKXHWpe5gIe6m9,1681183625.000000,gpt-3.5-turbo-0301,66.000000,"def fibonacci(n): """""" Computes nth Fibonacci number. """""" if n <= 1: return n else: return (fibonacci(n - 1) + fibonacci(n - 2))def zero_pad(s): """""" Returns string s padded with zeros to make it 10 characters. """""" return s.zfill(10)"


In [23]:
# this is a PromptLogs object again. We'll add the response_parser to the logs
logs.append(response_parser)

## And we have another exchange

In [24]:
message_list = f"""
system: {sys_prompt_string}
user: Write a python function that returns numbers 1-10 in a list. 
"""

message_parser = MessageParser(message_list)
messages = message_parser.parse_messages()
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)
response_parser = ResponseParser(response=response)

logs.append(response_parser)

In [25]:
logs.df

,id,created,model,usage,all_code,code_blocks,content
0,chatcmpl-73ywrIJ5cfirs5nwKXHWpe5gIe6m9,1.681184e+09,gpt-3.5-turbo-0301,66.0,"def&nbsp;fibonacci(n):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;Computes&nbsp;nth&nbsp;Fibonacci&nbsp;number.<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;if&nbsp;n&nbsp;<=&nbsp;1:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;n<br>&nbsp;&nbsp;&nbsp;&nbsp;else:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;(fibonacci(n&nbsp;-&nbsp;1)&nbsp;+&nbsp;fibonacci(n&nbsp;-&nbsp;2))<br>def&nbsp;zero_pad(s):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;Returns&nbsp;string&nbsp;s&nbsp;padded&nbsp;with&nbsp;zeros&nbsp;to&nbsp;make&nbsp;it&nbsp;10&nbsp;characters.<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;s.zfill(10)<br>","[\ndef fibonacci(n):\n """"""\n Computes nth Fibonacci number.\n """"""\n if n <= 1:\n return n\n else:\n return (fibonacci(n - 1) + fibonacci(n - 2))\n, \ndef zero_pad(s):\n """"""\n Returns string s padded with zeros to make it 10 characters.\n """"""\n return s.zfill(10)\n]","Fibonacci sequence code:\n```python\ndef fibonacci(n):\n """"""\n Computes nth Fibonacci number.\n """"""\n if n <= 1:\n return n\n else:\n return (fibonacci(n - 1) + fibonacci(n - 2))\n```\n\nCode for zero-padding a string to 10 characters:\n```python\ndef zero_pad(s):\n """"""\n Returns string s padded with zeros to make it 10 characters.\n """"""\n return s.zfill(10)\n```\n\nThese functions could be saved in a Python file named `my_functions.py` for future use."
0,chatcmpl-73ywxpWRLcU2BzzKGkaMBluPZBmqn,1.681184e+09,gpt-3.5-turbo-0301,60.0,"def&nbsp;get_numbers():<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;[num&nbsp;for&nbsp;num&nbsp;in&nbsp;range(1,11)]<br>from&nbsp;utilities&nbsp;import&nbsp;get_numbers<br>","[\ndef get_numbers():\n return [num for num in range(1,11)]\n, \nfrom utilities import get_numbers\n]","The function to return numbers 1-10 in a list would look like this:\n\n```python\ndef get_numbers():\n return [num for num in range(1,11)]\n```\n\nHere, we have defined a function called `get_numbers` which simply returns a list containing numbers from 1 to 10. We're using a list comprehension to create and return the list. Note that we specify the range(1, 11), as it includes all values from 1 to 10. \n\nYou can store this function in a file called `utilities.py`. Of course, you could also use another name for the file as well. You'd just need to use the appropriate file name when you import the function elsewhere in your project. The import statement would look something like this:\n\n```python\nfrom utilities import get_numbers\n```"


### And print the styled logs with the code from two separate prompt exchanges

In [26]:
logs.styled_log

,created,model,usage,all_code
id,,,,
chatcmpl-73ywrIJ5cfirs5nwKXHWpe5gIe6m9,1681183625.000000,gpt-3.5-turbo-0301,66.000000,"def fibonacci(n): """""" Computes nth Fibonacci number. """""" if n <= 1: return n else: return (fibonacci(n - 1) + fibonacci(n - 2))def zero_pad(s): """""" Returns string s padded with zeros to make it 10 characters. """""" return s.zfill(10)"
chatcmpl-73ywxpWRLcU2BzzKGkaMBluPZBmqn,1681183631.000000,gpt-3.5-turbo-0301,60.000000,"def get_numbers(): return [num for num in range(1,11)]from utilities import get_numbers"
